<a href="https://colab.research.google.com/github/tvpian/Kafka-Stream-Processing/blob/main/docs%20/tutorials/Kafka_Anomaly_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/rodrigo-arenas/kafkaml-anomaly-detection.git

Cloning into 'kafkaml-anomaly-detection'...
remote: Enumerating objects: 175, done.
remote: Counting objects: 100% (175/175), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 175 (delta 88), reused 134 (delta 55), pack-reused 0
Receiving objects: 100% (175/175), 9.91 MiB | 9.58 MiB/s, done.
Resolving deltas: 100% (88/88), done.


In [2]:
cd /content/kafkaml-anomaly-detection

/content/kafkaml-anomaly-detection


In [3]:
!pip install kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 2.3 MB/s eta 0:00:00


In [55]:
!pip install slackclient==2.9.3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 1.2 MB/s eta 0:00:00


In [4]:
import os
from datetime import datetime
import time
import threading
import json
from kafka import KafkaProducer, KafkaConsumer
from kafka.errors import KafkaError
from sklearn.model_selection import train_test_split
import pandas as pd

In [5]:
!curl -sSOL https://dlcdn.apache.org/kafka/3.5.2/kafka_2.13-3.5.2.tgz
!tar -xzf kafka_2.13-3.5.2.tgz

In [6]:
!./kafka_2.13-3.5.2/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-3.5.2/config/zookeeper.properties
!./kafka_2.13-3.5.2/bin/kafka-server-start.sh -daemon ./kafka_2.13-3.5.2/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

Waiting for 10 secs until kafka and zookeeper services are up and running


In [7]:
!ps -ef | grep kafka

root        1430     307  0 21:17 ?        00:00:00 /bin/bash -c ps -ef | grep kafka
root        1432    1430  0 21:17 ?        00:00:00 grep kafka


In [8]:
# !./kafka_2.13-3.5.2/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic susy-train
# !./kafka_2.13-3.5.2/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 2 --topic susy-test

In [9]:
!./kafka_2.13-3.5.2/bin/kafka-topics.sh --bootstrap-server 127.0.0.1:9092 --topic transactions --create --partitions 3 --replication-factor 1

Created topic transactions.


In [10]:
!./kafka_2.13-3.5.2/bin/kafka-topics.sh --bootstrap-server 127.0.0.1:9092 --topic anomalies --create --partitions 3 --replication-factor 1

Created topic anomalies.


In [11]:
from joblib import dump
import numpy as np
from sklearn.ensemble import IsolationForest

rng = np.random.RandomState(42)
# Generate train data
X = 0.3 * rng.randn(500, 2)
X_train = np.r_[X + 2, X - 2]
X_train = np.round(X_train, 3)

# fit the model
clf = IsolationForest(n_estimators=50, max_samples=500, random_state=rng, contamination=0.01)
clf.fit(X_train)

dump(clf, '/content/isolation_forest.joblib')

['/content/isolation_forest.joblib']

In [12]:

def error_callback(exc):
    raise Exception('Error while sendig data to kafka: {0}'.format(str(exc)))

def write_to_kafka(topic_name, items):
  count=0
  producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'])
  for message, key in items:
    producer.send(topic_name, key=key.encode('utf-8'), value=message.encode('utf-8')).add_errback(error_callback)
    count+=1
  producer.flush()
  print("Wrote {0} messages into topic: {1}".format(count, topic_name))

# write_to_kafka("susy-train", zip(x_train, y_train))
# write_to_kafka("susy-test", zip(x_test, y_test))

In [13]:
TRANSACTIONS_TOPIC = "transactions"
OUTLIERS_GENERATION_PROBABILITY = 0.7
DELAY = 0.01
TRANSACTIONS_CONSUMER_GROUP = "transactions"
NUM_PARTITIONS = 3
ANOMALIES_TOPIC = "anomalies"
ANOMALIES_CONSUMER_GROUP = "anomalies"

In [113]:
import json
import random
import time
from datetime import datetime
import numpy as np


_id = 0
producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'])

if producer is not None:
    while _id<1000:
        # Generate some abnormal observations
        if random.random() <= OUTLIERS_GENERATION_PROBABILITY:
            X_test = np.random.uniform(low=-4, high=4, size=(1, 2)).tolist()
        else:
            X = 0.3 * np.random.randn(1, 2)
            X_test = (X + np.random.choice(a=[2, -2], size=1, p=[0.5, 0.5]))
            X_test = np.round(X_test, 3).tolist()

        current_time = datetime.utcnow().isoformat()
        # print(record)
        record = {"id": _id, "data": X_test, "current_time": current_time}
        record = json.dumps(record).encode("utf-8")

        producer.send(topic=TRANSACTIONS_TOPIC,
                         value=record,
                         key=str(_id).encode('utf-8')).add_errback(error_callback)
        producer.flush()
        _id += 1
        time.sleep(DELAY)

In [114]:
!./kafka_2.13-3.5.2/bin/kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --topic transactions  --from-beginning --max-messages 10

{"id": 4, "data": [[2.390535874343068, -1.2254359178294578]], "current_time": "2024-04-27T21:17:34.733387"}
{"id": 6, "data": [[1.324, 2.052]], "current_time": "2024-04-27T21:17:34.765197"}
{"id": 10, "data": [[-3.9188534028559454, -0.28411030390906333]], "current_time": "2024-04-27T21:17:34.829552"}
{"id": 12, "data": [[0.19288200132585231, 2.2585051160358303]], "current_time": "2024-04-27T21:17:34.858077"}
{"id": 13, "data": [[-1.4810164790612053, -3.7441448489875464]], "current_time": "2024-04-27T21:17:34.872787"}
{"id": 14, "data": [[-2.149, -2.262]], "current_time": "2024-04-27T21:17:34.886839"}
{"id": 18, "data": [[-1.930797526779779, 0.41818699171613005]], "current_time": "2024-04-27T21:17:34.947745"}
{"id": 19, "data": [[-1.864, -1.709]], "current_time": "2024-04-27T21:17:34.962502"}
{"id": 20, "data": [[2.231, 2.479]], "current_time": "2024-04-27T21:17:34.977267"}
{"id": 24, "data": [[-1.507221089499657, -0.9159198044566459]], "current_time": "2024-04-27T21:17:35.033694"}
Proc

In [115]:
# from kafka import KafkaConsumer


# # Kafka consumer configuration
# consumer = KafkaConsumer(TRANSACTIONS_TOPIC,
#                          bootstrap_servers=['127.0.0.1:9092'],
#                          group_id=TRANSACTIONS_CONSUMER_GROUP,
#                          auto_offset_reset='earliest')

# try:
#     for msg in consumer:
#         # Print the message key and value
#         print('Received message: {}'.format(msg.value.decode('utf-8')))

#         # Check if there are no more messages to read
#         if msg is None:
#             print("No more messages to read. Exiting.")
#             break

# except KeyboardInterrupt:
#     # Close the consumer on keyboard interrupt
#     consumer.close()

In [116]:
# from kafka import KafkaConsumer


# # Kafka consumer configuration
# consumer = KafkaConsumer(TRANSACTIONS_TOPIC,
#                          bootstrap_servers=['127.0.0.1:9092'],
#                          group_id=TRANSACTIONS_CONSUMER_GROUP,
#                          auto_offset_reset='earliest')

# try:
#     # Set a timeout for the consumer
#     consumer_timeout_ms = 10000  # 1 second

#     while True:
#         # Poll for new messages with timeout
#         msg_batch = consumer.poll(timeout_ms=consumer_timeout_ms)

#         if msg_batch:
#             for partition, messages in msg_batch.items():
#                 for msg in messages:
#                     # Print the message key and value
#                     print('Received message: {}'.format(msg.value.decode('utf-8')))
#         else:
#             # No more messages to read. Exiting.
#             print("No more messages to read. Exiting.")
#             break

# except KeyboardInterrupt:
#     # Close the consumer on keyboard interrupt
#     consumer.close()
# consumer.close()

In [117]:
from kafka import KafkaConsumer


# Kafka consumer configuration
consumer = KafkaConsumer(TRANSACTIONS_TOPIC,
                         bootstrap_servers=['127.0.0.1:9092'],
                         group_id=TRANSACTIONS_CONSUMER_GROUP,
                         auto_offset_reset='earliest', enable_auto_commit=False,
                         max_poll_records=5000)

try:
    # Set a timeout for the consumer
    consumer_timeout_ms = 1000  # 1 second

    while True:
        # Poll for new messages with timeout
        msg_batch = consumer.poll(timeout_ms=consumer_timeout_ms)

        if msg_batch:
            for partition, messages in msg_batch.items():
                for msg in messages:
                    # Print the message key and value
                    print('Received message: {}'.format(msg.value.decode('utf-8')))
                    ###############################################################
                    ############### Prediction on the recieved data################
                    record = json.loads(msg.value.decode('utf-8'))
                    data = record["data"]
                    prediction = clf.predict(data)
                    ########### Send the prediction if it is an Anomaly###########
                    if prediction[0] == -1:
                      score = clf.score_samples(data)
                      record["score"] = np.round(score, 3).tolist()
                      _id = record["id"]
                      record = json.dumps(record).encode("utf-8")
                      print(record)
                      producer.send(topic=ANOMALIES_TOPIC,
                                      value=record,
                                      key=str(_id).encode('utf-8')).add_errback(error_callback)
                      producer.flush()
        else:
            # No more messages to read. Exiting.
            print("No more messages to read. Exiting.")
            break

except KeyboardInterrupt:
    # Close the consumer on keyboard interrupt
    consumer.close()
consumer.close()

Received message: {"id": 1, "data": [[-1.895, -1.95]], "current_time": "2024-04-27T23:11:51.074679"}
Received message: {"id": 5, "data": [[3.5048077625721916, 2.4865230983251054]], "current_time": "2024-04-27T23:11:51.126466"}
b'{"id": 5, "data": [[3.5048077625721916, 2.4865230983251054]], "current_time": "2024-04-27T23:11:51.126466", "score": [-0.697]}'
Received message: {"id": 7, "data": [[2.0687548321927993, -3.694216656622607]], "current_time": "2024-04-27T23:11:51.152406"}
b'{"id": 7, "data": [[2.0687548321927993, -3.694216656622607]], "current_time": "2024-04-27T23:11:51.152406", "score": [-0.712]}'
Received message: {"id": 8, "data": [[-0.4401089652411896, -2.2958198207194043]], "current_time": "2024-04-27T23:11:51.165577"}
b'{"id": 8, "data": [[-0.4401089652411896, -2.2958198207194043]], "current_time": "2024-04-27T23:11:51.165577", "score": [-0.659]}'
Received message: {"id": 11, "data": [[-3.951647508138495, -0.22917820159472058]], "current_time": "2024-04-27T23:11:51.204641"

In [118]:
!./kafka_2.13-3.5.2/bin/kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --topic anomalies  --from-beginning --max-messages 10

{"id": 0, "data": [[-0.7133644614349457, 3.535406214653851]], "current_time": "2024-04-27T21:46:21.805071", "score": [-0.745]}
{"id": 16, "data": [[-0.24878916812103213, -2.438986432404323]], "current_time": "2024-04-27T21:46:22.013110", "score": [-0.68]}
{"id": 32, "data": [[3.1712160721093285, -1.9736686238579786]], "current_time": "2024-04-27T21:46:22.214815", "score": [-0.715]}
{"id": 36, "data": [[0.6014426467433314, 1.8392335893421619]], "current_time": "2024-04-27T21:46:22.265216", "score": [-0.686]}
{"id": 40, "data": [[2.9168678735091094, 1.4289823852236587]], "current_time": "2024-04-27T21:46:22.315368", "score": [-0.705]}
{"id": 41, "data": [[3.4304602183443995, -0.8018763569537608]], "current_time": "2024-04-27T21:46:22.327781", "score": [-0.758]}
{"id": 57, "data": [[2.0876218141203733, -3.709253970577663]], "current_time": "2024-04-27T21:46:22.529998", "score": [-0.719]}
{"id": 69, "data": [[-3.0743796404138424, -3.5613348634919513]], "current_time": "2024-04-27T21:46:22.

In [119]:
SLACK_API_TOKEN = "" # Fill with your token name
SLACK_CHANNEL = "kafka_anomaly_detection" # Replace with your custom channel name


In [120]:
import logging
from slack import WebClient
from slack.errors import SlackApiError

client = WebClient(token=SLACK_API_TOKEN)


In [121]:
response = client.chat_postMessage(channel=SLACK_CHANNEL, text="SASA")

In [122]:
from kafka import KafkaConsumer


# Kafka consumer configuration
consumer = KafkaConsumer(TRANSACTIONS_TOPIC,
                         bootstrap_servers=['127.0.0.1:9092'],
                         group_id=TRANSACTIONS_CONSUMER_GROUP,
                         auto_offset_reset='earliest', enable_auto_commit=False,
                         max_poll_records=5000)

try:
    # Set a timeout for the consumer
    consumer_timeout_ms = 1000  # 1 second

    while True:
        # Poll for new messages with timeout
        msg_batch = consumer.poll(timeout_ms=consumer_timeout_ms)

        if msg_batch:
            for partition, messages in msg_batch.items():
                for msg in messages:
                    # Print the message key and value
                    print('Received message: {}'.format(msg.value.decode('utf-8')))
                    ###############################################################
                    ############### Prediction on the recieved data################
                    record = json.loads(msg.value.decode('utf-8'))
                    try:
                        # Send message to slack channel
                        response = client.chat_postMessage(channel=SLACK_CHANNEL,
                                                          text=record)
                    except SlackApiError as e:

                        print(e.response["error"])

                    consumer.commit()
        else:
            # No more messages to read. Exiting.
            print("No more messages to read. Exiting.")
            break

except KeyboardInterrupt:
    # Close the consumer on keyboard interrupt
    consumer.close()
consumer.close()

Received message: {"id": 0, "data": [[-1.443, -2.027]], "current_time": "2024-04-27T23:11:51.056507"}
Received message: {"id": 2, "data": [[-3.611029882583698, -0.6550538652631586]], "current_time": "2024-04-27T23:11:51.087738"}
Received message: {"id": 3, "data": [[-0.7795881373643763, 1.9413692866595023]], "current_time": "2024-04-27T23:11:51.100697"}
Received message: {"id": 9, "data": [[2.6800085646945586, 0.24386787957483147]], "current_time": "2024-04-27T23:11:51.178575"}
Received message: {"id": 16, "data": [[1.72, 2.367]], "current_time": "2024-04-27T23:11:51.270857"}
Received message: {"id": 29, "data": [[-1.981, -1.873]], "current_time": "2024-04-27T23:11:51.443249"}
Received message: {"id": 32, "data": [[-3.352484048517919, -0.2920481698616353]], "current_time": "2024-04-27T23:11:51.482449"}
Received message: {"id": 36, "data": [[2.7687662965543742, -3.174072196155082]], "current_time": "2024-04-27T23:11:51.534355"}
Received message: {"id": 40, "data": [[-1.5830652941316776,

In [ ]:



client = WebClient(token=SLACK_API_TOKEN)
# Kafka consumer configuration
consumer = KafkaConsumer(ANOMALIES_TOPIC,
                         bootstrap_servers=['127.0.0.1:9092'],
                         group_id=ANOMALIES_CONSUMER_GROUP,
                         auto_offset_reset='earliest', enable_auto_commit=False,
                         max_poll_records=5000)

while True:
    message = consumer.poll()
    if message is None:
        continue
    if message.error():
        logging.error("Consumer error: {}".format(message.error()))
        continue

    # Message that came from producer
    record = message.value().decode('utf-8')

    try:
        # Send message to slack channel
        response = client.chat_postMessage(channel=SLACK_CHANNEL,
                                           text=record)
    except SlackApiError as e:

        print(e.response["error"])

    consumer.commit()

consumer.close()